In [42]:
import json
import csv
import os
#
def extract_standard_w_h(standard_file):
    data_w_h = {}
    with open(standard_file, 'r') as file:
        for line in file:
            parts = line.strip().split(';')
            if len(parts) < 2:
                continue            
            key = parts[0].strip()
            value = int(parts[1].strip())
            data_w_h[key] = value
    return data_w_h
#
def extract_design_data(file_path):
    # Extract design data from design files
    # Same function to extract data from *.sim, *.cal or json file
    data = {}
    if file_path.endswith('.json'):
        with open(file_path, "r") as json_file:
            data = json.load(json_file)
            return data
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            # skip empty lines or comments
            if not line or line.startswith('#') or line.startswith('$'):
                continue
            separator = "=" # In casee of sim file data with = as separator
            if ';' in line: # In casee of cal file data with ; as separator
                separator = ";"
            # check if line contains 'separator' and then split
            if separator in line:
                key, value = line.split(separator)
                key = key.strip()
                value = value.strip()
                data[key] = value
    return data
#
def convert_design_to_cal_format(design_file):
    file_name_no_extension = os.path.splitext(design_file)[0]
    out_design_file = file_name_no_extension + "_converted.cal"
    input_data = extract_design_data(design_file)
    with open(out_design_file, "w") as f:
        for key, value in input_data.items():
            f.write(f"{key} ; {value}\n")
    return (out_design_file)
#
def convert_design_to_sim_format(design_file):
    file_name_no_extension = os.path.splitext(design_file)[0]
    out_design_file = file_name_no_extension + "_converted.sim"
    input_data = extract_design_data(design_file)
    with open(out_design_file, "w") as f:
        for key, value in input_data.items():
            f.write(f"{key} = {value}\n")
    return (out_design_file)
#
def convert_design_to_neutral_json_format(design_file):
    file_name_no_extension = os.path.splitext(design_file)[0]
    out_design_file = file_name_no_extension + "_converted.json"
    input_data = extract_design_data(design_file)
    with open(out_design_file, "w") as json_file:
        json.dump(input_data, json_file, indent=0)
    return (out_design_file)
#
def get_IPE_80_600_steel_beam_data(filename):
    """
    There are well defined allowed design for I-Section steel beams
    IPE = I-Section Profile European standard
    Example: IPE 80: has height 80 mm and other dimensions are mentioned in PDF/CSV
    Reads an IPE beam CSV file (semicolon-separated) and returns
    a dictionary with 'h (mm)' as key and other numeric columns as list.
    Ignores the first column and the header row.
    """
    ipe_dict = {}
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')
        next(reader)  # Skip header row
        for row in reader:
            if not row:  # skip empty rows
                continue
            try:
                key = int(row[1])  # second column: h = height (mm)
                values = [int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6])]
                #values = [width, moment_of_inertia, area, min_length, max_Length]
                ipe_dict[key] = values
            except (ValueError, IndexError):
                # Skip malformed or incomplete rows safely
                continue
    return ipe_dict
#
# Deflection Calculation for IPE Steel Sections
def ipe_steel_deflection(load, span, I, E=200000, load_type='point'):
    """
    Calculate deflection for simply supported IPE beam.
    
    Parameters:
    load: Load in N (point) or N/mm (distributed)
    span: Span length in mm
    I: Moment of inertia in mm⁴
    E: Modulus of elasticity in N/mm² (default 200000)
    load_type: 'point' or 'distributed'
    """
    if load_type == 'point':
        return (load * span**3) / (48 * E * I)
    else:  # distributed
        return (5 * load * span**4) / (384 * E * I)
#
def create_history_data(history_file):
    header = (
    "Design file Name;L (mm);Type;Material;Shape;"
    "h (mm);w (mm);F (N);V (mm^3);Deflection (mm);"
    "Allowable_Def (mm) L/240;Def_Ratio %;Status;Reason"
    )
    file = open(history_file, "w")
    file.write(header + "\n")
    F = 10000 # Default applied force hardcoded
    ######################## Prepare Steel I-Section Beams Data ################
    E = 200000  # N/mm²
    ipe_data = get_IPE_80_600_steel_beam_data("IPE_Steel_Beam_Dimensions.csv")
    for h, val in ipe_data.items():
        w = val[0]
        moment_of_inertia = val[1] # for deflection
        area = val[2] # for volume
        desgin_name = "" #5000_Beam_Steel_IPE_100_50_10000.json
        #for L in range(start_m, end_m + 1, 1000):  # iterate in meters
        for L in range(1000, 20001, 500):  # iterate in meters    
            design_name = str(L)+"_Beam_Steel_IPE_"+str(h)+"_"+str(w)+"_"+str(F)+".json"
            V = L * area
            allowed_def = L/240 # Standard
            deflection = ipe_steel_deflection(F, L, moment_of_inertia, E, 'point')
            percent_deflection = round(100*deflection/allowed_def)
            allowed_def_r = round(allowed_def)
            deflection_r = round(deflection)
            file.write(f"{design_name};{L};Beam;Steel;IPE;{h};{w};{F};{V};{deflection_r};{allowed_def_r};")
            file.write(f"{percent_deflection};")
            if percent_deflection >=100:
                file.write(f"FAIL;Excessive Deflection: {deflection_r} mm and Allowable Deflection = {allowed_def_r} mm\n")
            else:
                file.write(f"PASS; Deflection: {deflection_r} mm < Allowable Deflection = {allowed_def_r} mm\n")
    ######################## Prepare Wood Data ################
    standard_wood_file = "Standard_Wood_Beam_Diemnsions.csv"
    standard_wood_data = extract_standard_w_h(standard_wood_file)
    print("Wood Standard:" + str(standard_wood_data))
    min_width = standard_wood_data["min_width"]
    max_width = standard_wood_data["max_width"]
    min_height = standard_wood_data["min_height"]
    max_height = standard_wood_data["max_height"]    
    E = 10000  # N/mm²
    # Start from min_width, increase by 10 (e.g., 40, 50, 60)
    for w in range(min_width, max_width +1, 10):
        # Second value runs from min_height, step = 10
        for h in range(min_height, max_height + 1, 10):
            if h<w:
                continue
            if h>4*w:
                continue
            for L in range(1000, 10001, 500):
                design_name = str(L)+"_Beam_Wood_Rectangular_"+str(h)+"_"+str(w)+"_"+str(F)+".json"
                V = L * w * h
                allowed_def = L/240 # Standard
                moment_of_inertia = (w * h**3) / 12  # mm⁴
                deflection = (F * L**3) / (48 * E * moment_of_inertia)
                percent_deflection = round(100*deflection/allowed_def)
                allowed_def_r = round(allowed_def)
                deflection_r = round(deflection)
                file.write(f"{design_name};{L};Beam;Wood;Rectangular;{h};{w};{F};{V};{deflection_r};{allowed_def_r};")
                file.write(f"{percent_deflection};")
                if percent_deflection >=100:
                    file.write(f"FAIL;Excessive Deflection: {deflection_r} mm and Allowable Deflection = {allowed_def_r} mm\n")
                else:
                    file.write(f"PASS; Deflection: {deflection_r} mm < Allowable Deflection = {allowed_def_r} mm\n")                                
    ######################## Concrete Data ################
    standard_concrete_file = "Standard_Concrete_Beam_Diemnsions.csv"
    standard_concrete_data = extract_standard_w_h(standard_concrete_file)
    print("Concrete Standard:" + str(standard_concrete_data))
    min_width = standard_concrete_data["min_width"]
    max_width = standard_concrete_data["max_width"]
    min_height = standard_concrete_data["min_height"]
    max_height = standard_concrete_data["max_height"]    
    E = 25000  # N/mm²
    # Start from 50, increase by 10 (e.g., 50, 60, 70)
    for w in range(min_width, max_width + 1, 10):
        # Second value runs from w to 2*w, step = 20
        for h in range(min_height, max_height + 1, 10):
            if h<w:
                continue
            if h>w*5:
                continue
            for L in range(1000, 15001, 500):
                design_name = str(L)+"_Beam_Concrete_Rectangular_"+str(h)+"_"+str(w)+"_"+str(F)+".json"
                V = L * w * h
                allowed_def = L/240 # Standard
                moment_of_inertia = (w * h**3) / 12  # mm⁴
                deflection = (F * L**3) / (48 * E * moment_of_inertia)
                percent_deflection = round(100*deflection/allowed_def)
                allowed_def_r = round(allowed_def)
                deflection_r = round(deflection)
                file.write(f"{design_name};{L};Beam;Concrete;Rectangular;{h};{w};{F};{V};{deflection_r};{allowed_def_r};")
                file.write(f"{percent_deflection};")
                if percent_deflection >=100:
                    file.write(f"FAIL;Excessive Deflection: {deflection_r} mm and Allowable Deflection = {allowed_def_r} mm\n")
                else:
                    file.write(f"PASS; Deflection: {deflection_r} mm < Allowable Deflection = {allowed_def_r} mm\n")                                
    ######################## Data Creation ended ################
    file.close()
    return header
#
history_file = "extracted_historical_data_00.csv"
#print(create_history_data(history_file))
#print("Historical data created")
def calculate_rect_def(Length, Width, Height,E_mod):
    E_mod = 25000
    Force = 10000
    moment_of_inertia = Width * Height**3/12
    deflection = (Force * Length**3) / (48 * E_mod * moment_of_inertia)
    return int(deflection)
def created_all_optimized_designs(best_history_designs):
    lines = []
    with open(best_history_designs, 'r') as file:
        lines = file.readlines()
    header = lines.pop(0)
    print(header)
    for line in lines:
        parts = line.strip().split(';')
        #Design file Name;L (mm);Type;Material;Shape;h (mm);w (mm);F (N);V (mm^3);
        #Deflection (mm);Allowable_Def (mm) L/240;Def_Ratio %;Status;Reason
        Length = int(parts[1])
        Material = parts[3]
        Shape = parts[4]
        if "IPE" in Shape:
            continue
        E_mod = 10000
        if Material == "Concrete":
            E_mod = 25000            
        Height = int(parts[5])
        Width = int(parts[6])
        Force = int(parts[7])
        Volume = int(parts[8])
        Deflection = int(parts[9])
        print("def: " + str(Deflection))
        calculated_def = calculate_rect_def(Length, Width, Height, E_mod)
        print("Calculated deflection = " + str(calculated_def))
        Allowable_Def = int(parts[10])
        Def_Ratio = int(parts[11])
        print(Allowable_Def)
        Status = parts[12]
        print(Status)

        #print(shape)
    
#
best_history_designs = "extracted_historical_data_BEST_00.csv"
created_all_optimized_designs(best_history_designs)
######

Design file Name;L (mm);Type;Material;Shape;h (mm);w (mm);F (N);V (mm^3);Deflection (mm);Allowable_Def (mm) L/240;Def_Ratio %;Status;Reason

def: 0
Calculated deflection = 0
4
PASS
def: 4
Calculated deflection = 1
4
PASS
def: 0
Calculated deflection = 0
6
PASS
def: 6
Calculated deflection = 2
6
PASS
def: 1
Calculated deflection = 0
8
PASS
def: 8
Calculated deflection = 3
8
PASS
def: 1
Calculated deflection = 1
10
PASS
def: 10
Calculated deflection = 3
10
PASS
def: 2
Calculated deflection = 2
12
PASS
def: 12
Calculated deflection = 4
12
PASS
def: 4
Calculated deflection = 3
15
PASS
def: 13
Calculated deflection = 5
15
PASS
def: 5
Calculated deflection = 5
17
PASS
def: 17
Calculated deflection = 6
17
PASS
def: 8
Calculated deflection = 7
19
PASS
def: 19
Calculated deflection = 7
19
PASS
def: 10
Calculated deflection = 10
21
PASS
def: 20
Calculated deflection = 8
21
PASS
def: 14
Calculated deflection = 13
23
PASS
def: 21
Calculated deflection = 8
23
PASS
def: 18
Calculated deflection = 18